In [15]:
import sys
import numpy as np
import random
import pandas as pd
sys.path.append('dna2vec/dna2vec/dna2vec/')

In [16]:
from multi_k_model import MultiKModel

In [17]:
import sys
print(sys.version)

3.6.7 |Anaconda custom (64-bit)| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [18]:
def getSequenceEmbeddingRandomKMer(model,sequence,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):     
        limit = min(8,len(currentSequence))
        kmer = random.randint(3,limit,random_state=21)
        evaluateSequence = currentSequence[:kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[kmer:]        
    return embeddingVector        

In [19]:
def getSequenceEmbeddingWithKMer(model,sequence,kmer,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):             
        chosen_kmer = min(len(currentSequence),kmer)   
        if((len(currentSequence) - kmer) < 3 and (len(currentSequence) - kmer)>0):
            chosen_kmer = chosen_kmer -  (3-(len(currentSequence) - kmer))   
        evaluateSequence = currentSequence[:chosen_kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[chosen_kmer:]            
    return embeddingVector      

In [20]:
def getSequenceEmbeddingWithKMerLow(model,sequence,kmer,embeddingLenght):
    currentSequence = sequence.replace("N","")    
    embeddingVector = [0.0]*embeddingLenght
    while(currentSequence):             
        chosen_kmer = min(len(currentSequence),kmer)   
        if((len(currentSequence) - kmer) < 3 and (len(currentSequence) - kmer)>0):
            chosen_kmer = len(currentSequence)   
        evaluateSequence = currentSequence[:chosen_kmer]
        embeddingVector += model.vector(evaluateSequence)
        currentSequence = currentSequence[chosen_kmer:]            
    return embeddingVector      

In [21]:
train_filepath = "train_workspace/"
filepath_model = "dna_complete/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Intermediate","Relaxed"]
dataList = ["Ohnologs","No-Ohnologs","Paralogs"]
dataNameList = ["ohnologs-complete_p2.pkl","no-ohnologs-complete_p2.pkl","paralog-complete_p2.pkl"]

In [22]:
def get_df_dict(level):
    df_dict = {}
    df_dict["Ohnologs"] = {}
    df_dict["No-Ohnologs"] = {}
    df_dict["Paralogs"] = {}
    current_file_path = train_filepath + level + "/python2_pickle/"
    for (dataName,data) in zip(dataNameList,dataList):                
        df_dict[data] = pd.read_pickle(current_file_path + dataName)        
    return df_dict

In [23]:
kmer = 3

In [24]:
models_dict = {}
models_dict["Strict"] = {}
models_dict["Intermediate"] = {}
models_dict["Relaxed"] = {}
for level in levelList:
    for animal in animalList:
        model_file_path = filepath_model + "dna_" + animal.lower() + "/results/" + animal.lower() + "_model.w2v"
        models_dict[level][animal] = MultiKModel(model_file_path)    

In [25]:
len(models_dict["Intermediate"]["Human"].vector("AAA"))

100

In [26]:
df_embedding_dict = {}
for level in levelList:
    df_dict = get_df_dict(level)    
    df_embedding_dict[level] = {}
    for data in dataList:                    
        embeddings_sequence_1 = []
        embeddings_sequence_2 = []        
        for index, row in df_dict[data].iterrows():
            model = models_dict[level][row["Species"]]
            embeddings_sequence_1.append(getSequenceEmbeddingWithKMerLow(model,row["Sequence-1"],kmer,100))
            embeddings_sequence_2.append(getSequenceEmbeddingWithKMerLow(model,row["Sequence-2"],kmer,100))
        
        df_embedding_dict[level][data] = []
        df_embeddings_1 = pd.DataFrame(embeddings_sequence_1)        
        df_embeddings_1.columns = ['Embedding1_' + str(i) for i in range(0,100)]                               
        
        df_embeddings_2 = pd.DataFrame(embeddings_sequence_2)        
        df_embeddings_2.columns = ['Embedding2_' + str(i) for i in range(0,100)]                
        
        df_embedding_dict[level][data].append(df_embeddings_1)
        df_embedding_dict[level][data].append(df_embeddings_2)

In [27]:
for level in levelList:    
    for data in dataList:    
        df_embedding_dict[level][data][0].to_pickle(train_filepath + level + "/embeddings_dna2vec/" + data.lower() + "1-vec-"+ str(kmer) + "kmer.pkl")
        df_embedding_dict[level][data][1].to_pickle(train_filepath + level + "/embeddings_dna2vec/" + data.lower() + "2-vec-"+ str(kmer) + "kmer.pkl")

In [28]:
df_embedding_dict["Intermediate"]["Ohnologs"][0].head()

,Embedding1_0,Embedding1_1,Embedding1_2,Embedding1_3,Embedding1_4,Embedding1_5,Embedding1_6,Embedding1_7,Embedding1_8,Embedding1_9,...,Embedding1_90,Embedding1_91,Embedding1_92,Embedding1_93,Embedding1_94,Embedding1_95,Embedding1_96,Embedding1_97,Embedding1_98,Embedding1_99
0,-71.816959,153.487438,-120.695948,-126.544171,-2.467456,199.958826,77.557535,73.452212,-12.583375,53.166530,...,244.810385,14.050933,-31.651109,87.134815,-103.104198,-16.837504,-29.319812,10.368657,82.795033,236.893929
1,-116.226763,225.788995,-197.514149,-192.971279,-18.977980,369.727333,157.189840,133.468697,-2.264011,87.339197,...,411.645689,58.578911,-96.291944,148.850147,-184.388376,-36.750970,-31.205902,50.920799,140.715709,346.059986
2,-287.606741,517.279894,-421.204400,-367.805686,-19.308499,691.425038,409.279641,266.193189,-8.508934,221.893021,...,983.051001,116.551606,-187.440621,214.602453,-309.010096,-66.494229,-92.759825,151.622433,162.232729,697.768576
3,-68.113295,120.420032,-111.212594,-140.716395,-6.598510,245.269009,79.645608,75.331922,-3.474544,58.424476,...,170.776587,40.221223,-0.081509,88.906888,-97.706538,-7.051719,-5.515214,-11.457295,133.202463,263.723514
4,-42.780687,137.596874,-116.930844,-118.933972,-14.710021,217.883033,78.006250,63.965493,6.827861,53.131663,...,223.218364,33.566448,-50.593383,93.460978,-97.559199,-12.892246,-10.896256,14.975375,80.779956,202.175052
